# Import

In [1]:
import os
import cv2
import csv
import math
import numpy as np
import pandas as pd
import mediapipe as mp
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.signal import savgol_filter

# Global Variables

In [13]:
KeypointsROOT = "data/"
VIDEOROOT = "data/"

TIMESTAMP_PATH = "_timeStamp_0916.csv" ### [update] 2023/09/16
VIDEOTYPE_PATH = "_videoType.csv"

invalidList = ['60', '61', '85']

In [3]:
LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398]
RIGHT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246]

LEFT_IRIS = [474, 475, 476, 477] ### 右, 上, 左, 下
RIGHT_IRIS = [469, 470, 471, 472] ### 右, 上, 左, 下

LEFT_EYEBROW =[336, 296, 334, 293, 300, 276, 283, 282, 295, 285]
RIGHT_EYEBROW = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46]

COLOR_BLACK = (0, 0, 0)
COLOR_WHITE = (255, 255, 255)
COLOR_RED = (0, 0, 255)
COLOR_GREEN = (0, 255, 0)
COLOR_BLUE = (255, 0, 0)
COLOR_PINK = (147,20,255)
COLOR_YELLOW = (0,255,255)
COLOR_CYAN = (255,255,0)
COLOR_MAGENTA = (255,0,255)
COLOR_GRAY = (128,128,128)
COLOR_PURPLE = (128,0,128)
COLOR_ORANGE = (0,165,255)

# GPU Setting

In [4]:
import tensorflow as tf

print(tf.__version__)
print(tf.config.list_physical_devices())

2.5.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
      #  for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


# Load Timestamp

In [6]:
timeStamp_dict = {}

def load_timestamp():
    if not os.path.exists(TIMESTAMP_PATH):
        print(f'[Timestamp] CSV does not exit.')
    else:
        df = pd.read_csv(TIMESTAMP_PATH)
        df = df.fillna(0)
        n_row, n_col = df.shape[0], df.shape[1]
        
        data_dict = {}
        for idx in range(n_row):
            pid = int(df.iloc[idx, 0])
            start1 = int(float(df.iloc[idx, 1]))
            end1 = int(float(df.iloc[idx, 2]))
            
            if start1==0 and end1==0:
                continue
            else:
                data_dict[pid] = {}
                data_dict[pid]["1-start"] = start1
                data_dict[pid]["1-end"] = end1
    return data_dict


timeStamp_dict = load_timestamp()

In [7]:
print(f'=== number of pid in timeStamp CSV file: {len(timeStamp_dict)} ===\n')
# print(f'Without pid: 41, 96, 97, 98, 99, 100\n')
print(f'Without pid: 41\n') ### [update] 2023/09/16

for key in timeStamp_dict:
    print(f'key:{key}, {timeStamp_dict[key]}')

=== number of pid in timeStamp CSV file: 99 ===

Without pid: 41

key:1, {'1-start': 67, '1-end': 119}
key:2, {'1-start': 111, '1-end': 149}
key:3, {'1-start': 62, '1-end': 96}
key:4, {'1-start': 111, '1-end': 150}
key:5, {'1-start': 119, '1-end': 168}
key:6, {'1-start': 57, '1-end': 79}
key:7, {'1-start': 62, '1-end': 84}
key:8, {'1-start': 93, '1-end': 110}
key:9, {'1-start': 61, '1-end': 75}
key:10, {'1-start': 106, '1-end': 128}
key:11, {'1-start': 145, '1-end': 160}
key:12, {'1-start': 73, '1-end': 109}
key:13, {'1-start': 71, '1-end': 193}
key:14, {'1-start': 59, '1-end': 79}
key:15, {'1-start': 66, '1-end': 108}
key:16, {'1-start': 91, '1-end': 139}
key:17, {'1-start': 85, '1-end': 137}
key:18, {'1-start': 90, '1-end': 118}
key:19, {'1-start': 84, '1-end': 148}
key:20, {'1-start': 71, '1-end': 101}
key:21, {'1-start': 64, '1-end': 95}
key:22, {'1-start': 87, '1-end': 118}
key:23, {'1-start': 82, '1-end': 118}
key:24, {'1-start': 80, '1-end': 231}
key:25, {'1-start': 71, '1-end':

# Load ID

In [8]:
def loadID(KeypointsROOT):
    tmp_id = []
    for pid in list(os.listdir(KeypointsROOT)):
        if int(pid) in timeStamp_dict:
            tmp_id.append(pid)
        if (int(pid) in timeStamp_dict)==False:
            print(f'pid [{int(pid)}] not in timeStamp_dict.')
    return tmp_id
    
allID_list = loadID(KeypointsROOT)

In [9]:
print(f'=== number of valid pid: {len(allID_list)} subjects. ===\n')

print(allID_list)

=== number of valid pid: 99 subjects. ===

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '100', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']


# Load Video Type

In [10]:
videoType_dict = {}

def load_videotype():
    if not os.path.exists(VIDEOTYPE_PATH):
        print(f'[VideoType] CSV does not exit.')
    else:
        df = pd.read_csv(VIDEOTYPE_PATH)
        n_row, n_col = df.shape[0], df.shape[1]
        
        data_dict = {}
        for idx in range(n_row):
            pid = int(df.iloc[idx, 0])
            videotype = int(df.iloc[idx, 1])
            
            data_dict[pid] = {}
            data_dict[pid]["type"] = videotype
    return data_dict

    
videoType_dict = load_videotype()

In [11]:
print(f'Number of subjects in videoType_dict: {len(videoType_dict)} subjects.\n')

print(videoType_dict)

Number of subjects in videoType_dict: 99 subjects.

{1: {'type': 1}, 2: {'type': 1}, 3: {'type': 1}, 4: {'type': 1}, 5: {'type': 1}, 6: {'type': 1}, 7: {'type': 1}, 8: {'type': 1}, 9: {'type': 1}, 10: {'type': 1}, 100: {'type': 2}, 11: {'type': 1}, 12: {'type': 1}, 13: {'type': 1}, 14: {'type': 1}, 15: {'type': 1}, 16: {'type': 1}, 17: {'type': 1}, 18: {'type': 1}, 19: {'type': 1}, 20: {'type': 1}, 21: {'type': 1}, 22: {'type': 1}, 23: {'type': 1}, 24: {'type': 1}, 25: {'type': 1}, 26: {'type': 1}, 27: {'type': 1}, 28: {'type': 1}, 29: {'type': 1}, 30: {'type': 1}, 31: {'type': 1}, 32: {'type': 1}, 33: {'type': 1}, 34: {'type': 1}, 35: {'type': 1}, 36: {'type': 1}, 37: {'type': 1}, 38: {'type': 1}, 39: {'type': 1}, 40: {'type': 1}, 42: {'type': 1}, 43: {'type': 1}, 44: {'type': 1}, 45: {'type': 1}, 46: {'type': 1}, 47: {'type': 1}, 48: {'type': 1}, 49: {'type': 1}, 50: {'type': 1}, 51: {'type': 1}, 52: {'type': 1}, 53: {'type': 2}, 54: {'type': 2}, 55: {'type': 2}, 56: {'type': 2}, 57:

# Z-axis data
* Pid: 27, 20, 9

In [26]:
invalidID_list = []

def landmarks_detection_z_axis(image, results, isDraw=False):
    imgHeight, imgWidth= image.shape[:2]
    # list[(x,y), (x,y)....]: the list of tuples for each landmarks 
    mesh_coord = [(int(point.x * imgWidth), int(point.y * imgHeight), point.z) for point in results.multi_face_landmarks[0].landmark]
    if isDraw:
        [cv2.circle(image, p, 2, COLOR_GREEN, -1) for p in mesh_coord]
    return mesh_coord

def get_z_value(coords, img_width):
    n_landmarks = len(coords)
    ### [Left] Eye = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
    pt1_z = coords[398][2]*img_width
    pt2_z = coords[362][2]*img_width
    pt3_z = coords[382][2]*img_width
    pt4_z = coords[374][2]*img_width
    pt5_z = coords[249][2]*img_width
    pt6_z = coords[263][2]*img_width
    pt7_z = coords[466][2]*img_width
    pt8_z = coords[386][2]*img_width
    
    ### [Right] Eye = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246]
    pt9_z = coords[173][2]*img_width
    pt10_z = coords[133][2]*img_width
    pt11_z = coords[155][2]*img_width
    pt12_z = coords[145][2]*img_width
    pt13_z = coords[7][2]*img_width
    pt14_z = coords[33][2]*img_width
    pt15_z = coords[246][2]*img_width
    pt16_z = coords[159][2]*img_width
    
    data = [pt1_z, pt2_z, pt3_z, pt4_z, pt5_z, pt6_z, pt7_z, pt8_z, pt9_z, pt10_z, pt11_z, pt12_z, pt13_z, pt14_z, pt15_z, pt16_z]
    return data

def get_z_axis_data(pidList):
    mp_face_mesh = mp.solutions.face_mesh
    with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
        for pid in pidList:
            video_path = VIDEOROOT + pid + '/Description1/clip1.mp4'
            csv_output_path = VIDEOROOT + pid + '/Description1_20240705/z_axis_data.csv'

            if os.path.exists(VIDEOROOT + pid + '/Description1_20240705/')==False:
                os.mkdir(VIDEOROOT + pid + '/Description1_20240705/')

            if not os.path.exists(video_path):
                print(f'====== Pid [{pid}] clip1 does NOT exist. ======')
            else:
                print(f'[Pid {pid}] {video_path}, {csv_output_path}')

                ### Load Video [Description 1] ------------------------------------------------------
                cap = cv2.VideoCapture(video_path)
                n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                frameWidth, frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                FPS = cap.get(cv2.CAP_PROP_FPS)

                cnt_no_face = 0
                canWriteCSV = True
                dataList = []
                for frame in tqdm(range(n_frames), total=n_frames):
                    ret, image = cap.read()
                    if ret == False:
                        break
                    results = face_mesh.process(image)
                    if results.multi_face_landmarks:
                        meshCoord = landmarks_detection_z_axis(image, results, False)
                        # data = get_coordinate_data(meshCoord)
                        img_height, img_width= image.shape[:2]
                        data = get_z_value(meshCoord, img_width)
                        dataList.append(data)
                    else:
                        cnt_no_face += 1
                        if cnt_no_face > 10*30 and canWriteCSV==True: ### 10 seconds
                            invalidID_list.append(pid)
                            print(f'[Info] pid {pid} - no face detected.')
                            canWriteCSV = False
                            break
                cap.release()
                cv2.destroyAllWindows()
                if canWriteCSV == True:
                    with open(csv_output_path, 'w', newline='') as outfile:
                        csv_writer = csv.writer(outfile)
                        csv_writer.writerows(dataList)


# subject_id_list = ['09', '20', '27'] ### AD
# subject_id_list = ['90', '05', '43'] ### Non-AD
subject_id_list = allID_list ### Run all subjects
get_z_axis_data(subject_id_list)

[Pid 01] data/01/Description1/clip1.mp4, data/01/Description1_20240705/z_axis_data.csv


100%|██████████| 1590/1590 [00:18<00:00, 84.31it/s]


[Pid 02] data/02/Description1/clip1.mp4, data/02/Description1_20240705/z_axis_data.csv


100%|██████████| 1170/1170 [00:13<00:00, 84.30it/s]


[Pid 03] data/03/Description1/clip1.mp4, data/03/Description1_20240705/z_axis_data.csv


...

100%|██████████| 1860/1860 [00:25<00:00, 74.20it/s]


[Pid 98] data/98/Description1/clip1.mp4, data/98/Description1_20240705/z_axis_data.csv


100%|██████████| 2370/2370 [00:31<00:00, 74.72it/s]


[Pid 99] data/99/Description1/clip1.mp4, data/99/Description1_20240705/z_axis_data.csv


100%|██████████| 2580/2580 [00:34<00:00, 74.32it/s]


## Smooth

In [27]:
def smooth_z_axis_data(pidList):
    for pid in pidList:
        if pid not in invalidList:
            # if pid=='27':
            if pid >= '01':
                csv_file = VIDEOROOT + pid + '/Description1_20240705/z_axis_data.csv'
                csv_file_output = VIDEOROOT + pid + '/Description1_20240705/z_axis_data_smooth.csv'
    
                if not os.path.exists(csv_file):
                    print(f'[{pid}] csv file does not exist.')
                else:
                    df = pd.read_csv(csv_file, header=None)
                    n_row, n_col = df.shape[0], df.shape[1]

                    resultArray_smooth = []
                    for i in range(n_col):
                        x = df.iloc[:, i].values
                        x_smooth = savgol_filter(x, window_length=21, polyorder=15)
                        x_smooth = np.round(x_smooth, 2)
                        resultArray_smooth.append(x_smooth)

                        # plt.figure(figsize=(15, 5))
                        # plt.plot(x, 'r')
                        # plt.plot(x_smooth, 'b')
                        # plt.show()

                    smooth_data = []
                    resultArray = np.array(resultArray_smooth)
                    nFrames = resultArray.shape[1]
                    for i in range(nFrames):
                        smooth_data.append([resultArray[0][i], resultArray[1][i], resultArray[2][i], resultArray[3][i],
                                            resultArray[4][i], resultArray[5][i], resultArray[6][i], resultArray[7][i],
                                            resultArray[8][i], resultArray[9][i], resultArray[10][i], resultArray[11][i], 
                                            resultArray[12][i], resultArray[13][i], resultArray[14][i], resultArray[15][i]
                                            ])
                    smooth_data = np.array(smooth_data)
                    df_smooth = pd.DataFrame(smooth_data)
                    print(f'[Pid {pid}] {csv_file_output}')
                    df_smooth.to_csv(csv_file_output, index=False, header=False)


# subject_id_list = ['09', '20', '27'] ### AD
# subject_id_list = ['90', '05', '43'] ### Non-AD
subject_id_list = allID_list ### Run all subjects
smooth_z_axis_data(subject_id_list)

[Pid 01] data/01/Description1_20240705/z_axis_data_smooth.csv
[Pid 02] data/02/Description1_20240705/z_axis_data_smooth.csv
[Pid 03] data/03/Description1_20240705/z_axis_data_smooth.csv
[Pid 04] data/04/Description1_20240705/z_axis_data_smooth.csv
[Pid 05] data/05/Description1_20240705/z_axis_data_smooth.csv
[Pid 06] data/06/Description1_20240705/z_axis_data_smooth.csv
[Pid 07] data/07/Description1_20240705/z_axis_data_smooth.csv
[Pid 08] data/08/Description1_20240705/z_axis_data_smooth.csv
[Pid 09] data/09/Description1_20240705/z_axis_data_smooth.csv
[Pid 10] data/10/Description1_20240705/z_axis_data_smooth.csv
[Pid 100] data/100/Description1_20240705/z_axis_data_smooth.csv
[Pid 11] data/11/Description1_20240705/z_axis_data_smooth.csv
[Pid 12] data/12/Description1_20240705/z_axis_data_smooth.csv
[Pid 13] data/13/Description1_20240705/z_axis_data_smooth.csv
[Pid 14] data/14/Description1_20240705/z_axis_data_smooth.csv
[Pid 15] data/15/Description1_20240705/z_axis_data_smooth.csv
[Pid 1

# New Time Series data

In [28]:
def get_timeseries_data_with_zaxis(pidList):
    for pid in pidList:
        if pid not in invalidList:
            # if pid=='27':
            if pid >= '01':
                csv_file_xy = VIDEOROOT + pid + '/Description1_20240314/3_timeSeries_smooth.csv'
                csv_file_z = VIDEOROOT + pid + '/Description1_20240705/z_axis_data_smooth.csv'
                csv_file_output = VIDEOROOT + pid + '/Description1_20240705/timeSeries_zaxis.csv'
    
                if not os.path.exists(csv_file_xy) or not os.path.exists(csv_file_z):
                    print(f'[{pid}] csv file does not exist.')
                    break
                else:
                    df = pd.read_csv(csv_file_xy, header=None)
                    df_z = pd.read_csv(csv_file_z, header=None)
                    
                    lEye_pt1_x, lEye_pt2_x, lEye_pt3_x = [], [], []
                    lEye_pt1_y, lEye_pt2_y, lEye_pt3_y = [], [], []
                    lEye_pt1_z, lEye_pt2_z, lEye_pt3_z = [], [], []
                    rEye_pt1_x, rEye_pt2_x, rEye_pt3_x = [], [], []
                    rEye_pt1_y, rEye_pt2_y, rEye_pt3_y = [], [], []
                    rEye_pt1_z, rEye_pt2_z, rEye_pt3_z = [], [], []

                    for index, row in df_z.iterrows():
                        left_inside = (row[0]+row[1]+row[2])/3
                        left_middle = (row[3]+row[7])/2
                        left_outside = (row[4]+row[5]+row[6])/3
                        right_inside = (row[8]+row[9]+row[10])/3
                        right_middle = (row[11]+row[15])/2
                        right_outside = (row[12]+row[13]+row[14])/3
                        lEye_pt1_z.append(round(left_inside, 2))
                        lEye_pt2_z.append(round(left_middle, 2))
                        lEye_pt3_z.append(round(left_outside, 2))
                        rEye_pt1_z.append(round(right_inside, 2))
                        rEye_pt2_z.append(round(right_middle, 2))
                        rEye_pt3_z.append(round(right_outside, 2))

                    n_row, n_col = df.shape[0], df.shape[1]
                    for i in range(n_row):
                        left_inside_x, left_inside_y = df.iloc[[i], [0]].values[0][0], df.iloc[[i], [1]].values[0][0]
                        left_middle_x, left_middle_y = df.iloc[[i], [2]].values[0][0], df.iloc[[i], [3]].values[0][0]
                        left_outside_x, left_outside_y = df.iloc[[i], [4]].values[0][0], df.iloc[[i], [5]].values[0][0]
                        right_inside_x, right_inside_y = df.iloc[[i], [6]].values[0][0], df.iloc[[i], [7]].values[0][0]
                        right_middle_x, right_middle_y = df.iloc[[i], [7]].values[0][0], df.iloc[[i], [9]].values[0][0]
                        right_outside_x, right_outside_y = df.iloc[[i], [10]].values[0][0], df.iloc[[i], [11]].values[0][0]

                        lEye_pt1_x.append(round(left_inside_x, 2))
                        lEye_pt2_x.append(round(left_middle_x, 2))
                        lEye_pt3_x.append(round(left_outside_x, 2))
                        lEye_pt1_y.append(round(left_inside_y, 2))
                        lEye_pt2_y.append(round(left_middle_y, 2))
                        lEye_pt3_y.append(round(left_outside_y, 2))

                        rEye_pt1_x.append(round(right_inside_x, 2))
                        rEye_pt2_x.append(round(right_middle_x, 2))
                        rEye_pt3_x.append(round(right_outside_x, 2))
                        rEye_pt1_y.append(round(right_inside_y, 2))
                        rEye_pt2_y.append(round(right_middle_y, 2))
                        rEye_pt3_y.append(round(right_outside_y, 2))


                    feature_name_list = ['lEye_pt1_x', 'lEye_pt1_y', 'lEye_pt1_z',
                                         'lEye_pt2_x', 'lEye_pt2_y', 'lEye_pt2_z', 
                                         'lEye_pt3_x', 'lEye_pt3_y', 'lEye_pt3_z', 
                                         'rEye_pt1_x', 'rEye_pt1_y', 'rEye_pt1_z', 
                                         'rEye_pt2_x', 'rEye_pt2_y', 'rEye_pt2_z', 
                                         'rEye_pt3_x', 'rEye_pt3_y', 'rEye_pt3_z']
                    coord_value_list = [lEye_pt1_x, lEye_pt1_y, lEye_pt1_z,
                                        lEye_pt2_x, lEye_pt2_y, lEye_pt2_z, 
                                        lEye_pt3_x, lEye_pt3_y, lEye_pt3_z, 
                                        rEye_pt1_x, rEye_pt1_y, rEye_pt1_z, 
                                        rEye_pt2_x, rEye_pt2_y, rEye_pt2_z, 
                                        rEye_pt3_x, rEye_pt3_y, rEye_pt3_z]
                    
                    df_new = pd.DataFrame()
                    for featureIdx, feature in enumerate(feature_name_list):
                        df_new.insert(featureIdx, feature, coord_value_list[featureIdx])
                    print(f'[Pid {pid}] output: {csv_file_output}')
                    df_new.to_csv(csv_file_output, index=None, header=None)



# subject_id_list = ['09', '20', '27'] ### AD
# subject_id_list = ['90', '05', '43'] ### Non-AD
subject_id_list = allID_list ### Run all subjects
get_timeseries_data_with_zaxis(subject_id_list)

[Pid 01] output: data/01/Description1_20240705/timeSeries_zaxis.csv
[Pid 02] output: data/02/Description1_20240705/timeSeries_zaxis.csv
[Pid 03] output: data/03/Description1_20240705/timeSeries_zaxis.csv
[Pid 04] output: data/04/Description1_20240705/timeSeries_zaxis.csv
[Pid 05] output: data/05/Description1_20240705/timeSeries_zaxis.csv
[Pid 06] output: data/06/Description1_20240705/timeSeries_zaxis.csv
[Pid 07] output: data/07/Description1_20240705/timeSeries_zaxis.csv
[Pid 08] output: data/08/Description1_20240705/timeSeries_zaxis.csv
[Pid 09] output: data/09/Description1_20240705/timeSeries_zaxis.csv
[Pid 10] output: data/10/Description1_20240705/timeSeries_zaxis.csv
[Pid 100] output: data/100/Description1_20240705/timeSeries_zaxis.csv
[Pid 11] output: data/11/Description1_20240705/timeSeries_zaxis.csv
[Pid 12] output: data/12/Description1_20240705/timeSeries_zaxis.csv
[Pid 13] output: data/13/Description1_20240705/timeSeries_zaxis.csv
[Pid 14] output: data/14/Description1_20240705

# Show Z-axis on the video
* https://github.com/google-ai-edge/mediapipe/blob/master/docs/solutions/face_mesh.md

In [47]:
def show_z_axis_on_video(pidList):
    for pid_str in pidList:
        pid = int(pid_str)
        vtype = videoType_dict[pid]['type']
        if vtype==1:
            IMG_HEIGHT = 480
            IMG_WIDTH = 640
        elif vtype==2:
            IMG_HEIGHT = 720
            IMG_WIDTH = 1280

        my_FPS = 10
        z_axis_csv_file = "data/" + pid_str + "/Description1_20240705/z_axis_data.csv"
        video_input = "data/" + pid_str + "/Description1/clip1.mp4"
        video_output = "data/" + pid_str + "/Description1_20240705/clip1_with_z_axis_FPS" + str(my_FPS) + ".mp4"

        ### Read z-axis data from CSV file
        df = pd.read_csv(z_axis_csv_file, header=None)
        print(f'[pid: {pid}], {z_axis_csv_file}, videoType: {vtype}')
        n_row, n_col = df.shape[0], df.shape[1]
        sum_z = df.sum(axis = 1).tolist()
        avg_z = [x/16 for x in sum_z]
        scaled_z = [x*IMG_WIDTH for x in avg_z]
        # print(len(scaled_z))

        ### Get video info
        cap = cv2.VideoCapture(video_input)
        FPS = cap.get(cv2.CAP_PROP_FPS)

        ### Define video writer
        VIDEO_CODEC = "mp4v"
        OUT_WIDTH, OUT_HEIGHT = IMG_WIDTH, IMG_HEIGHT
        # video_writer = cv2.VideoWriter(video_output, cv2.VideoWriter_fourcc(*VIDEO_CODEC), FPS, (IMG_WIDTH, IMG_HEIGHT))
        video_writer = cv2.VideoWriter(video_output, cv2.VideoWriter_fourcc(*VIDEO_CODEC), my_FPS, (IMG_WIDTH, IMG_HEIGHT))

        n_video_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        for frameIdx, frame in enumerate(tqdm(range(n_video_frames), total=n_video_frames)):
            if frameIdx < len(scaled_z):
                ret, img = cap.read()
                if ret==False: break
                img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_NEAREST)
                dist_z = round(scaled_z[frameIdx], 2)

                img = cv2.putText(img, 'frameIdx: '+str(frameIdx), org=(50,100), color=(0,0,0), thickness=2, fontScale=1, fontFace=cv2.FONT_HERSHEY_SIMPLEX, lineType=cv2.LINE_AA)
                img = cv2.putText(img, 'depth: '+str(dist_z), org=(50,150), color=(0,0,0), thickness=2, fontScale=1, fontFace=cv2.FONT_HERSHEY_SIMPLEX, lineType=cv2.LINE_AA)
                video_writer.write(img)
    video_writer.release()
    cap.release()

# subject_id_list = ['09', '20', '27'] ### AD
# subject_id_list = ['90', '05', '43'] ### Non-AD
# subject_id_list = ['05']
subject_id_list = ['27']
show_z_axis_on_video(subject_id_list)

[pid: 27], data/27/Description1_20240705/z_axis_data.csv, videoType: 1


100%|██████████| 1230/1230 [00:02<00:00, 457.37it/s]
